In [13]:
import numpy as np
import awkward as ak
import fastjet
import h5py

def cluster_jets(event, jetdef, pt_min=20.0):
    
    event = ak.Array(event)
    print(event)
    
    cluster = fastjet.ClusterSequence(event, jetdef)    
    jets_ak = cluster.inclusive_jets(pt_min)
    jets = ak.to_numpy(jets_ak)

    #... get jet feature

    jet_pt = np.sqrt(jets['px']**2 + jets['py']**2)
    jet_rap = np.arctanh(jets['pz'] / jets['E'])
    jet_phi = np.arctan2(jets['py'], jets['px'])
    jet_M2 = jets['E']**2 - jets['px']**2 - jets['py']**2 - jets['pz']**2
    jet_M = np.sqrt(jet_M2[jet_M2 > 0])

    jets = np.array(list(zip(jet_pt, jet_rap, jet_phi, jet_M)))
    idx = np.argsort(-jets[0])  # The negative sign is to sort in descending order
    return jets[idx]

In [72]:

jet_def = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)
f = h5py.File('../../../../data/jetclass/qcd_top_jets/top_N30_100k.hdf5', 'r')
events=[]
for ev in list(f['4_momenta'][..., :4]):
    px, py, pz, e = ev[:, 0], ev[:, 1], ev[:, 2], ev[:, 3]
    event = ak.zip({"px": px, "py": py, "pz": pz, "E": e}, with_name="Momentum4D")
    events.append(event)



In [76]:
jetdef = fastjet.JetDefinition(fastjet.kt_algorithm,1)
fastjet.ClusterSequence(events[0], jetdef).inclusive_jets(20.0)

<Array [{px: -224, py: -492, pz: 920, ...}] type='1 * Momentum4D[px: float6...'>